import os
os.environ['PATH']
stream = os.popen('owltools --help')
output = stream.read()
output

In [1]:
import pandas as pd
# use ontobio!!

# Analysing ROBOT Report

from rdflib import Graph
g = Graph()
g.parse(mondo_file, format="xml")

from ontobio.ontol_factory import OntologyFactory
mondo_file="/Users/matentzn/ws/mondo/src/ontology/mondo-qc.obo"
ofa = OntologyFactory()
ont = ofa.create(mondo_file)
ont.get_roots()[:3]

In [2]:
report = 'obo-report.tsv'
df = pd.read_csv(report,sep='\t')
# Bug in ROBOT, wrong number of columns:
df.columns=['Rule Name', 'Subject', 'Property', 'Value','Trash']
df['Level'] = df.index
df.reset_index(level=0, inplace=True)
df

,index,Rule Name,Subject,Property,Value,Trash,Level
0,ERROR,duplicate_definition,MONDO:0001744,IAO:0000115,"The sudden increase of intraocular pressure, r...",NaN,ERROR
1,ERROR,duplicate_definition,MONDO:0001868,IAO:0000115,"The sudden increase of intraocular pressure, r...",NaN,ERROR
2,ERROR,duplicate_definition,HP:0032154,IAO:0000115,Oral aphthous ulcers typically present as pain...,NaN,ERROR
3,ERROR,duplicate_definition,MONDO:0005318,IAO:0000115,Oral aphthous ulcers typically present as pain...,NaN,ERROR
4,ERROR,duplicate_definition,CP:0000000,IAO:0000115,Cytoplasm that exhibits molecular interaction ...,NaN,ERROR
...,...,...,...,...,...,...,...
18300,INFO,lowercase_definition,http://purl.obolibrary.org/obo/uberon/core#tra...,IAO:0000115,.,NaN,INFO
18301,INFO,lowercase_definition,http://purl.obolibrary.org/obo/uberon/core#tra...,IAO:0000115,.,NaN,INFO
18302,INFO,lowercase_definition,http://purl.obolibrary.org/obo/uberon/core#tra...,IAO:0000115,.,NaN,INFO
18303,INFO,lowercase_definition,https://www.wikidata.org/wiki/Q2,IAO:0000115,third planet from the Sun in the Solar System,NaN,INFO


In [3]:
df=df[df['Subject'].str.startswith('MONDO')]
df[['Level','Rule Name','Value']].groupby(['Level','Rule Name']).count()

Value
Level Rule Name                         
ERROR duplicate_definition             4
      multiple_equivalent_classes      4
INFO  lowercase_definition           300
WARN  duplicate_exact_synonym       1688
      duplicate_scoped_synonym      1597
      missing_definition               0

In [4]:
df[df['Rule Name']=="lowercase_definition"]
# Revealed a more complex issue we need to sort first: https://github.com/monarch-initiative/mondo/issues/1979

,index,Rule Name,Subject,Property,Value,Trash,Level
17575,INFO,lowercase_definition,MONDO:0000712,IAO:0000115,frontotemporal dementia plus amyotrophic later...,NaN,INFO
17576,INFO,lowercase_definition,MONDO:0001554,IAO:0000115,secondary glaucoma caused by either excessive ...,NaN,INFO
17577,INFO,lowercase_definition,MONDO:0001709,IAO:0000115,sarcoidosis with a complication of hypercalcemia.,NaN,INFO
17578,INFO,lowercase_definition,MONDO:0001875,IAO:0000115,inflammation of the lateral epicondyle.,NaN,INFO
17579,INFO,lowercase_definition,MONDO:0002121,IAO:0000115,neuritis of a single nerve.,NaN,INFO
...,...,...,...,...,...,...,...
17870,INFO,lowercase_definition,MONDO:0025085,IAO:0000115,inflammation of the liver in animals due to vi...,NaN,INFO
17871,INFO,lowercase_definition,MONDO:0025100,IAO:0000115,inflammation of the udder in cows.,NaN,INFO
17872,INFO,lowercase_definition,MONDO:0044255,IAO:0000115,<Subhead> Genetic Heterogeneity of Variation i...,NaN,INFO
17873,INFO,lowercase_definition,MONDO:0044266,IAO:0000115,"{2,3:Berg and Bearn (1966, 1966)} discovered a...",NaN,INFO


In [5]:
df[df['Rule Name']=="deprecated_class_reference"]
# https://github.com/monarch-initiative/mondo/issues/1982

,index,Rule Name,Subject,Property,Value,Trash,Level


In [6]:
df[df['Rule Name']=="duplicate_definition"]
# https://github.com/monarch-initiative/mondo/issues/1975

,index,Rule Name,Subject,Property,Value,Trash,Level
0,ERROR,duplicate_definition,MONDO:0001744,IAO:0000115,"The sudden increase of intraocular pressure, r...",NaN,ERROR
1,ERROR,duplicate_definition,MONDO:0001868,IAO:0000115,"The sudden increase of intraocular pressure, r...",NaN,ERROR
3,ERROR,duplicate_definition,MONDO:0005318,IAO:0000115,Oral aphthous ulcers typically present as pain...,NaN,ERROR
7,ERROR,duplicate_definition,MONDO:0001014,IAO:0000115,A slowly progressing leukemia characterized by...,NaN,ERROR


In [7]:
df[df['Rule Name']=="duplicate_label"]
# https://github.com/monarch-initiative/mondo/issues/1972

,index,Rule Name,Subject,Property,Value,Trash,Level


In [8]:
df[df['Rule Name']=="multiple_equivalent_classes"]
# https://github.com/ontodev/robot/issues/733
# Ignore 0021108 0045024 until ticket resolved.

,index,Rule Name,Subject,Property,Value,Trash,Level
28,ERROR,multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,48ff0cdf-765b-4c84-b5f6-a66b87d09ecagenid324064,NaN,ERROR
29,ERROR,multiple_equivalent_classes,MONDO:0021108,owl:equivalentClass,48ff0cdf-765b-4c84-b5f6-a66b87d09ecagenid324068,NaN,ERROR
30,ERROR,multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,48ff0cdf-765b-4c84-b5f6-a66b87d09ecagenid350442,NaN,ERROR
31,ERROR,multiple_equivalent_classes,MONDO:0045024,owl:equivalentClass,48ff0cdf-765b-4c84-b5f6-a66b87d09ecagenid350446,NaN,ERROR


# Other checks analyses

In [9]:
report = 'single-child-warning.tsv'
df_scw = pd.read_csv(report,sep='\t') 
df_scw 
# Single child classes should be tagged. We should review them one by one.

,?cls,?total
0,<http://purl.obolibrary.org/obo/MONDO_0016094>,1
1,<http://purl.obolibrary.org/obo/MONDO_0021396>,1
2,<http://purl.obolibrary.org/obo/MONDO_0016093>,1
3,<http://purl.obolibrary.org/obo/MONDO_0008882>,1
4,<http://purl.obolibrary.org/obo/MONDO_0006224>,1
...,...,...
1554,<http://purl.obolibrary.org/obo/MONDO_0000461>,1
1555,<http://purl.obolibrary.org/obo/MONDO_0021540>,1
1556,<http://purl.obolibrary.org/obo/MONDO_0005846>,1
1557,<http://purl.obolibrary.org/obo/MONDO_0004648>,1


report = 'two-label-violation.tsv'
df_tlw = pd.read_csv(report,sep='\t') 
df_tlw

In [10]:
report = 'no-subclass-between-genetic-disease-warning.tsv'
df_nsbgd = pd.read_csv(report,sep='\t') 
df_nsbgd
## These cases should be tagged as well. But lets review them bit by bit


,?d1,?d1_label,?d2,?d2_label
0,<http://purl.obolibrary.org/obo/MONDO_0013529>,catecholaminergic polymorphic ventricular tach...,<http://purl.obolibrary.org/obo/MONDO_0008648>,"ventricular tachycardia, familial"
1,<http://purl.obolibrary.org/obo/MONDO_0013966>,catecholaminergic polymorphic ventricular tach...,<http://purl.obolibrary.org/obo/MONDO_0008648>,"ventricular tachycardia, familial"
2,<http://purl.obolibrary.org/obo/MONDO_0009597>,"metaphyseal chondrodysplasia, Spahr type",<http://purl.obolibrary.org/obo/MONDO_0009943>,Pyle disease
3,<http://purl.obolibrary.org/obo/MONDO_0012762>,catecholaminergic polymorphic ventricular tach...,<http://purl.obolibrary.org/obo/MONDO_0008648>,"ventricular tachycardia, familial"
4,<http://purl.obolibrary.org/obo/MONDO_0011211>,axial spondylometaphyseal dysplasia,<http://purl.obolibrary.org/obo/MONDO_0009943>,Pyle disease
...,...,...,...,...
63,<http://purl.obolibrary.org/obo/MONDO_0008477>,"spondylometaphyseal dysplasia, Kozlowski type",<http://purl.obolibrary.org/obo/MONDO_0009943>,Pyle disease
64,<http://purl.obolibrary.org/obo/MONDO_0010486>,"palmoplantar keratoderma, mutilating, with per...",<http://purl.obolibrary.org/obo/MONDO_0019014>,mutilating palmoplantar keratoderma with perio...
65,<http://purl.obolibrary.org/obo/MONDO_0020746>,"contractures, pterygia, and variable skeletal ...",<http://purl.obolibrary.org/obo/MONDO_0009926>,autosomal recessive multiple pterygium syndrome
66,<http://purl.obolibrary.org/obo/MONDO_0044304>,hyperphenylalaninemia due to DNAJC12 deficiency,<http://purl.obolibrary.org/obo/MONDO_0009861>,phenylketonuria


In [11]:
report = 'related-exact-synonym-warning.tsv'
df_res = pd.read_csv(report,sep='\t') 
df_res
# migrate provenance over after duplicate_exact_synonym check..
# Make tsv we should be strict
# label in one term should not be equal to exact synonym
# unique name should be unique

,?cls,?label,?related
0,<http://purl.obolibrary.org/obo/MONDO_0014756>,"tremor, hereditary essential, 5",ETM5
1,<http://purl.obolibrary.org/obo/MONDO_0006999>,tooth disease,disorder of calcareous tooth
2,<http://purl.obolibrary.org/obo/MONDO_0009424>,Bartter disease type 2,hyperprostaglandin E syndrome 2
3,<http://purl.obolibrary.org/obo/MONDO_0017686>,inborn aminoacylase deficiency,rare inborn error of aminoacylase activity
4,<http://purl.obolibrary.org/obo/MONDO_0020680>,acute bronchiolitis,acute bronchiolitis
...,...,...,...
195,<http://purl.obolibrary.org/obo/MONDO_0007068>,adenylosuccinate lyase deficiency,rare inborn error of (S)-2-(5-amino-1-(5-phosp...
196,<http://purl.obolibrary.org/obo/MONDO_0009626>,pseudo-TORCH syndrome,band-like calcification with simplified gyrati...
197,<http://purl.obolibrary.org/obo/MONDO_0003646>,rectum neuroendocrine neoplasm,rectum neuroendocrine tumor
198,<http://purl.obolibrary.org/obo/MONDO_0007435>,dentatorubral-pallidoluysian atrophy,DRPLA


In [12]:
report = 'excluded-subsumption-is-inferred.tsv'
df_esi = pd.read_csv(report,sep='\t') 
df_esi

,?cls,?label,?excluded,?excluded_label
0,<http://purl.obolibrary.org/obo/MONDO_0043174>,Pfeiffer Tietze Welte syndrome,<http://purl.obolibrary.org/obo/MONDO_0001071>,intellectual disability
1,<http://purl.obolibrary.org/obo/MONDO_0002722>,olfactory nerve neoplasm,<http://purl.obolibrary.org/obo/MONDO_0002722>,olfactory nerve neoplasm
2,<http://purl.obolibrary.org/obo/MONDO_0023059>,Elliott ludman Teebi syndrome,<http://purl.obolibrary.org/obo/MONDO_0001071>,intellectual disability
3,<http://purl.obolibrary.org/obo/MONDO_0024524>,dyschromatosis universalis hereditaria 1,<http://purl.obolibrary.org/obo/MONDO_0006600>,pigmentation disease
4,<http://purl.obolibrary.org/obo/MONDO_0018695>,avian influenza,<http://purl.obolibrary.org/obo/MONDO_0005087>,respiratory system disease
...,...,...,...,...
97,<http://purl.obolibrary.org/obo/MONDO_0044322>,intellectual developmental disorder with neuro...,<http://purl.obolibrary.org/obo/MONDO_0001071>,intellectual disability
98,<http://purl.obolibrary.org/obo/MONDO_0001650>,acute cystitis (disease),<http://purl.obolibrary.org/obo/MONDO_0006032>,cystitis
99,<http://purl.obolibrary.org/obo/MONDO_0001096>,mediastinum ganglioneuroblastoma,<http://purl.obolibrary.org/obo/MONDO_0005843>,mediastinal cancer
100,<http://purl.obolibrary.org/obo/MONDO_0043071>,Zazam Sheriff Phillips syndrome,<http://purl.obolibrary.org/obo/MONDO_0001071>,intellectual disability
